In [71]:
# Hack Sample Code
# This shows examples of logging in and using the subreddit, submission, redditor, ancd comments objects
# It is not implied that you need to collect these specific attributes of submissions or redditors, but are merely examples

# Imports
import praw
import pandas as pd
import numpy as np
import datetime
import math
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction import text


In [7]:

# Login and create Reddit instance
myClientIDvar = '7H-wkx3wmj96hJazvWQb-A'
myClientSecret = 'kKTtoMJA0gYXEr9d0ohMp-niRM0yhw'
myRedditUserName = 'Legitimate_Thing5210'
reddit = praw.Reddit(client_id=myClientIDvar, client_secret=myClientSecret, user_agent=myRedditUserName)


In [8]:
"""
Get the top 5 posts on specified subreddits
"""
subredditName = input('Name of subreddit: ')
subreddit_top_5 = reddit.subreddit(subredditName).hot(limit=5)

In [9]:
""""
Get Sample Dataframe of top 5 Posts' information
"""
post_lists = []

for indexPosts, submission in enumerate(subreddit_top_5):
    post_lists.append({
      'id': submission.id,
      'Sub': subredditName,
      'Title': submission.title,
      'URL': submission.permalink,
      'Time': datetime.datetime.fromtimestamp(submission.created_utc),
      'Author': str(submission.author),
      'Body': submission.selftext,
      'IsSelfPost': bool(submission.is_self)
    })

    print(submission.is_self)
    print('Retrieved data for '+str(indexPosts+1)+' posts')


dfPosts = pd.DataFrame(post_lists)
print(dfPosts)

False
Retrieved data for 1 posts
True
Retrieved data for 2 posts
False
Retrieved data for 3 posts
True
Retrieved data for 4 posts
False
Retrieved data for 5 posts
       id           Sub                                              Title  \
0  yaoef4  christianity  Hi! I think this Jesus plushie is very cute. W...   
1  yb1gn3  christianity                                Update on the baby!   
2  yag2y4  christianity  I hope this is allowed. I am severely bipolar ...   
3  yb3egc  christianity    Just a reminder to every Christian including me   
4  yaryd9  christianity  “Do not abandon yourselves to despair. We are ...   

                                                 URL                Time  \
0  /r/Christianity/comments/yaoef4/hi_i_think_thi... 2022-10-22 06:54:54   
1  /r/Christianity/comments/yb1gn3/update_on_the_... 2022-10-22 16:07:44   
2  /r/Christianity/comments/yag2y4/i_hope_this_is... 2022-10-21 23:25:17   
3  /r/Christianity/comments/yb3egc/just_a_reminde... 2022-10-22 

In [10]:
"""
Get Sample Dataframe of author information
"""


author_list = []
for indexAuthor, thisIndex in enumerate(dfPosts.index):
    # Get Entry Information
    thisSubmission = reddit.submission(id=dfPosts.loc[thisIndex,'id'])
    thisAuthor = thisSubmission.author

    # Get Author Information
    TempDictAuthor = {}

    # Verifies that author exists and has not been suspended (as suspended accounts do not have most attributes)
    if (thisSubmission.author is None) or not hasattr(thisAuthor, 'created_utc'):
        continue

    # Get data related to the author
    TempDictAuthor['userName'] = thisAuthor.name
    TempDictAuthor['numComments'] = sum(1 for _ in thisAuthor.comments.top(time_filter="month"))
    try:
        TempDictAuthor['upsGiven'] = sum(1 for _ in thisAuthor.upvoted())
    except:
        TempDictAuthor['upsGiven'] = np.nan

    try:
        TempDictAuthor['downsGiven'] = sum(1 for _ in thisAuthor.downvoted())
    except:
        TempDictAuthor['downsGiven'] = np.nan

    TempDictAuthor['awardsReceived'] = sum(1 for _ in thisAuthor.gilded()) # Need to sum all things in here
    print('Finished processing awardsReceived') # Debug
    try:
        TempDictAuthor['awardsGiven'] = sum(1 for _ in thisAuthor.gildings())
    except:
        TempDictAuthor['awardsGiven'] = np.nan

    TempDictAuthor['numComments'] = len(thisAuthor.trophies())
    TempDictAuthor['accountAge'] = thisAuthor.created_utc
    TempDictAuthor['totalKarma'] = thisAuthor.total_karma
    TempDictAuthor['linkKarma'] = thisAuthor.link_karma
    TempDictAuthor['commentKarma'] = thisAuthor.comment_karma
    TempDictAuthor['awarderKarma'] = thisAuthor.awarder_karma
    TempDictAuthor['awardeeKarma'] = thisAuthor.awardee_karma
    TempDictAuthor['isMod'] = thisAuthor.is_mod

    # Get Data related to posts by the author
    numPosts = 0
    numUpvotes = 0
    numUpDownRatio = 0 # Note: Cannot get Downvotes, only ratio of Up to Down
    numAwardsReceived = 0
    numCommentsList = []
    for thisAuthorPost in thisAuthor.submissions.top(time_filter="month"):
        numPosts = numPosts + 1
        numUpvotes = numUpvotes + thisAuthorPost.ups
        numUpDownRatio = numUpDownRatio + thisAuthorPost.upvote_ratio
        numAwardsReceived = numAwardsReceived + thisAuthorPost.total_awards_received
        for tempComment in thisAuthorPost.comments:
            if hasattr(tempComment,'is_submitter'): # If tempComment does not have a is_submitter attribute, it is because it is a "More Comments" link
                numComments = numComments + 1
            else:
                numComments = np.nan # For simplicity, just made this NaN as the summation is unknown unless clicking "More Comments", which is not part of this example code
        numCommentsList.append(numComments)

    TempDictAuthor['numPosts'] = numPosts
    TempDictAuthor['avgUpvotesReceived'] = np.divide(numUpvotes, numPosts)
    TempDictAuthor['avgUpToDownRatio'] = np.divide(numUpDownRatio, numPosts)
    TempDictAuthor['totalAwardsReceived'] = numAwardsReceived
    TempDictAuthor['avgAwardsPerPost'] = np.divide(numAwardsReceived, numPosts)
    TempDictAuthor['avgCommentsToPost'] = np.mean(numCommentsList)

    author_list.append(TempDictAuthor)

    print('Retrieved data for ' + str(indexAuthor + 1) + ' authors')

dfAuthors = pd.DataFrame(author_list)
dfAuthors['isMod'] = dfAuthors['isMod'].astype('bool')
print(dfAuthors)

laughayeette
Retrieved data for 1 authors
_mercurial_high_
Retrieved data for 2 authors
GirlWhoLovesGod
Retrieved data for 3 authors
stockbaron82
Retrieved data for 4 authors
usopsong
Retrieved data for 5 authors
           userName    accountAge  commentKarma  linkKarma  isMod
0      laughayeette  1.578487e+09           136        982  False
1  _mercurial_high_  1.636953e+09          1372       1939  False
2   GirlWhoLovesGod  1.654921e+09            26        336  False
3      stockbaron82  1.660429e+09            73        142  False
4          usopsong  1.388854e+09          6821      17310  False


In [11]:
"""
Get Sample Dataframe of top 5 Posts' information
"""
post_lists = []

for indexPosts, submission in enumerate(subreddit_top_5):
    post_lists.append({
      'id': submission.id,
      'Sub': subredditName,
      'Title': submission.title,
      'URL': submission.permalink,
      'Time': datetime.datetime.fromtimestamp(submission.created_utc),
      'Author': str(submission.author),
      'Body': submission.selftext,
      'IsSelfPost': bool(submission.is_self)
    })

    print(submission.is_self)
    print('Retrieved data for '+str(indexPosts+1)+' posts')


dfPosts = pd.DataFrame(post_lists)
print(dfPosts)

laughayeette
Retrieved data for 1 authors
_mercurial_high_
Retrieved data for 2 authors
GirlWhoLovesGod
Retrieved data for 3 authors
stockbaron82
Retrieved data for 4 authors
usopsong
Retrieved data for 5 authors
           userName    accountAge  commentKarma  linkKarma  isMod
0      laughayeette  1.578487e+09           136        982  False
1  _mercurial_high_  1.636953e+09          1372       1939  False
2   GirlWhoLovesGod  1.654921e+09            26        336  False
3      stockbaron82  1.660429e+09            73        142  False
4          usopsong  1.388854e+09          6821      17310  False


In [77]:
#df = pd.read_csv('./results/christianity_posts.csv')

#df
import math
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction import text

directory = 'results'

for filename in os.listdir(directory):

    if "_posts" in filename:

        # print("./results/" + filename)
        df = pd.read_csv("./results/" + filename)

        print(df)

"""
directory = 'results'
x = os.listdir(directory)
print(x)
"""

         ID             Author             Sub  \
0    y1d9oq    Jeremehthejelly             bjj   
1    yaseov     Mikethedeadman       Christian   
2    yastpf     Mikethedeadman           Bible   
3    yas9aq     Mikethedeadman     askanything   
4    yawd9f     thetruthiseeit           Bible   
..      ...                ...             ...   
123  y7fpb3            Animore           Bible   
124  xsib9l  user-234522467898  amateur_boxing   
125  yaepa4  user-234522467898    AtlantaHawks   
126  yaddol  user-234522467898    AtlantaHawks   
127  y79bl7  user-234522467898           Bible   

                                                 Title  \
0           Has anyone been to a Ruotolo bros seminar?   
1                                     Is hell forever?   
2                                     Is hell forever?   
3                                      Is hell forever   
4    The phrase "the land had peace 40 years" in Ju...   
..                                                 

"\ndirectory = 'results'\nx = os.listdir(directory)\nprint(x)\n"

In [75]:

my_stop_words = text.ENGLISH_STOP_WORDS

vectorizer = TfidfVectorizer(stop_words=my_stop_words)
X = vectorizer.fit_transform(df['Body'].fillna(''))

print(vectorizer.get_feature_names_out())
print(X)
print(X.shape)


['000' '10' '100' ... 'יו' 'ים' 'ἀββᾶ']
  (0, 558)	0.021693865001873057
  (0, 451)	0.016380115332650942
  (0, 591)	0.021693865001873057
  (0, 396)	0.021693865001873057
  (0, 1225)	0.021693865001873057
  (0, 1125)	0.019342493314762316
  (0, 756)	0.021693865001873057
  (0, 1423)	0.019342493314762316
  (0, 612)	0.021693865001873057
  (0, 743)	0.019342493314762316
  (0, 504)	0.021693865001873057
  (0, 672)	0.019342493314762316
  (0, 1285)	0.019342493314762316
  (0, 505)	0.021693865001873057
  (0, 259)	0.021693865001873057
  (0, 88)	0.021693865001873057
  (0, 156)	0.021693865001873057
  (0, 1409)	0.021693865001873057
  (0, 1196)	0.021693865001873057
  (0, 1020)	0.021693865001873057
  (0, 997)	0.019342493314762316
  (0, 155)	0.03534833710518625
  (0, 1357)	0.021693865001873057
  (0, 262)	0.03868498662952463
  (0, 1461)	0.03868498662952463
  :	:
  (28, 906)	0.24967933244981172
  (28, 204)	0.12483966622490586
  (28, 1073)	0.11678140278449883
  (28, 817)	0.12483966622490586
  (28, 1069)	0.10996

In [76]:

labels = vectorizer.get_feature_names_out()
labels.shape


(1473,)

In [49]:

print(X)
print(X.shape)


  (0, 1113)	0.12253857751091682
  (0, 1142)	0.12253857751091682
  (0, 1217)	0.12253857751091682
  (0, 209)	0.12253857751091682
  (0, 543)	0.0524364983584078
  (0, 262)	0.12253857751091682
  (0, 620)	0.07326956213241458
  (0, 640)	0.099833177395358
  (0, 1152)	0.07326956213241458
  (0, 667)	0.12253857751091682
  (0, 751)	0.06384596964832746
  (0, 1167)	0.099833177395358
  (0, 1086)	0.12253857751091682
  (0, 231)	0.12253857751091682
  (0, 663)	0.061224013222576444
  (0, 1080)	0.047112870961383224
  (0, 624)	0.12253857751091682
  (0, 1047)	0.047112870961383224
  (0, 555)	0.12253857751091682
  (0, 898)	0.12253857751091682
  (0, 704)	0.061224013222576444
  (0, 1218)	0.12253857751091682
  (0, 517)	0.12253857751091682
  (0, 957)	0.12253857751091682
  (0, 626)	0.12253857751091682
  :	:
  (29, 1177)	0.0422242198822045
  (29, 85)	0.05013944704637496
  (29, 75)	0.0553887119385035
  (29, 933)	0.06215619141134521
  (29, 321)	0.110777423877007
  (29, 250)	0.1433888360245872
  (29, 796)	0.07846189748

In [52]:
labels[0:100]

array(['10', '11', '12', '13', '14', '15', '16', '17', '18', '1870', '19',
       '20', '2019', '2020', '2021', '21', '22', '23', '249', '25', '251',
       '26', '27', '28', '2yrs', '30', '37', '38', '408', '42', '450',
       '49', '71', '_____', 'aaaaaaaa_co', 'able', 'about', 'abraham',
       'abundant', 'abuse', 'abusive', 'accept', 'accepted', 'accepting',
       'accepts', 'accidentally', 'accordance', 'according', 'act',
       'actually', 'administrator', 'advance', 'adversaries', 'advice',
       'affirmed', 'afford', 'after', 'afternoon', 'again', 'against',
       'age', 'alive', 'all', 'almost', 'alone', 'also', 'always', 'am',
       'amazing', 'an', 'and', 'another', 'answers', 'antoninus',
       'anxiety', 'any', 'anyone', 'anything', 'anyways', 'apart',
       'apologize', 'appeared', 'appearing', 'applies', 'approaching',
       'are', 'aren', 'arise', 'arms', 'around', 'arrived', 'art', 'as',
       'ask', 'asked', 'asking', 'asleep', 'astonished', 'at', 'attack'],

In [ ]:

"""
def find_common(list_of_posts):

    set_unique_words = set()
    list_unique_words = []

    for post in list_of_posts:

        post = post.split(" ")
        list_unique_words = list_unique_words.append(post)
        set_unique_words = set(post).union(set_unique_words)

    for x in list_unique_words:
        
        for y in x:
            


# for word in list_unique_words:
    



# perform term frequency and return a dictionary of term frequencies
def perform_tf(dict_words, list_of_words):
    
    # initialize dictionary of term frequencies
    # length of list of words
    dict_tf = {}
    list_of_words_count = len(list_of_words)

    for word, count in wordDict.items():
        dict_tf[word] = count / float(list_of_words_count)

    return dict_tf

# perform idf on a given list of posts
def perform_idf(list_of_posts):

    N = len(list_of_posts)
    
    # initialize dictionary of idf's
    dict_idf = dict.fromkeys(list_of_posts[0].keys(), 0)

    for post in list_of_posts:
        for word, val in post.items():
            if val > 0:
                dict_idf[word] += 1
    
    for word, val in idfDict.items():
        dict_idf[word] = math.log(N / float(val))

    return dict_idf

"""




In [12]:
dfAuthors


,userName,accountAge,commentKarma,linkKarma,isMod
0,laughayeette,1.578487e+09,136,982,False
1,_mercurial_high_,1.636953e+09,1372,1939,False
2,GirlWhoLovesGod,1.654921e+09,26,336,False
3,stockbaron82,1.660429e+09,73,142,False
4,usopsong,1.388854e+09,6821,17310,False
